##Preprocessing
In this notebook, we mainly do several steps:  
1. Merge the data  
2. Average the February data & replace the origin Feb and Jan data.  
3. Deal with the "Accumulated" indicators.

###Merge the data in different sheets
When we scrab the data from the website, we save different categories in different sheets, so we first merge into one sheet.

In [ ]:
import pandas as pd

file_path = 'NBS_Dataset.xls'

# Load every sheet
xls = pd.ExcelFile(file_path)

# Initialize DataFrame
all_data = pd.DataFrame()

#Loop through all sheets
for i, sheet_name in enumerate(xls.sheet_names):
    # Skip first 2 title rows
    data = pd.read_excel(xls, sheet_name=sheet_name, header=2)
    # Keep the column if sheet1
    if i == 0:
        all_data = data
    else:  # Skip column
        all_data = all_data.append(data.iloc[1:], ignore_index=True)

# index
merged_data = all_data.reset_index(drop=True)




<ipython-input-1-f3f2c2efa6f9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data.iloc[1:], ignore_index=True)
<ipython-input-1-f3f2c2efa6f9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data.iloc[1:], ignore_index=True)
<ipython-input-1-f3f2c2efa6f9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data.iloc[1:], ignore_index=True)
<ipython-input-1-f3f2c2efa6f9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data.iloc[1:], ignore_index=True)
<ipython-input-1-f3f2c2efa6f9>:19: FutureWarning: The frame.append method is

In [ ]:
print(merged_data.head)

<bound method NDFrame.head of                                             Indicators  Oct 2023  Sep 2023  \
0    Total Retail Sales of Consumer Goods, Current ...       NaN   39826.0   
1    Total Retail Sales of Consumer Goods, Accumula...       NaN  342107.0   
2    Total Retail Sales of Consumer Goods, Growth R...       NaN       5.5   
3    Total Retail Sales of Consumer Goods, Accumula...       NaN       6.8   
4    Retail Sales of Consumer Goods of Enterprises ...       NaN   15890.3   
..                                                 ...       ...       ...   
146  Online Retail Sales in Goods, Accumulated(100 ...       NaN   90435.0   
147  Online Retail Sales in Goods, Accumulated Grow...       NaN       8.9   
148  Online Retail Sales in Food, Accumulated Growt...       NaN      10.4   
149  Online Retail Sales in Clothing, Accumulated G...       NaN       9.6   
150  Online Retail Sales in Necessities, Accumulate...       NaN       8.5   

     Aug 2023  Jul 2023  Jun 2023

In [ ]:
merged_data.columns = merged_data.columns.map(str)
Feb_columns = [col for col in merged_data.columns if 'Feb' in col]

for col in Feb_columns:
    merged_data[col] = merged_data[col] /2

print(merged_data.columns)

Index(['Indicators', 'Oct 2023', 'Sep 2023', 'Aug 2023', 'Jul 2023',
       'Jun 2023', 'May 2023', 'Apr 2023', 'Mar 2023', 'Feb 2023', 'Jan 2023',
       'Dec 2022', 'Nov 2022', 'Oct 2022', 'Sep 2022', 'Aug 2022', 'Jul 2022',
       'Jun 2022', 'May 2022', 'Apr 2022', 'Mar 2022', 'Feb 2022', 'Jan 2022',
       'Dec 2021', 'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021', 'Jul 2021',
       'Jun 2021', 'May 2021', 'Apr 2021', 'Mar 2021', 'Feb 2021', 'Jan 2021',
       'Dec 2020', 'Nov 2020', 'Oct 2020', 'Sep 2020', 'Aug 2020', 'Jul 2020',
       'June 2020', 'May 2020', 'Apr 2020', 'Mar 2020', 'Feb 2020', 'Jan 2020',
       'Dec 2019', 'Nov 2019', 'Oct 2019', 'Sep 2019', 'Aug 2019', 'Jul 2019',
       'Jun 2019', 'May 2019', 'Apr 2019', 'Mar 2019', 'Feb 2019', 'Jan 2019'],
      dtype='object')


In [ ]:
merged_data.columns = merged_data.columns.map(str)

# Find all pairing clos for Jan and Feb
Feb_columns = [col for col in merged_data.columns if 'Feb' in col]
Jan_columns = ['Jan' + col.split('Feb')[1] for col in Feb_columns]

# Replace Jan data by Feb
for Feb_col, Jan_col in zip(Feb_columns, Jan_columns):
    if Jan_col in merged_data.columns:  # Replace only when Jan exists
        merged_data[Jan_col] = merged_data[Feb_col]


###Accumulated Problem
When calculating average for Feb index and replace Jan & Feb by the avg, there might be a problem of the indicators showing the accumulated data. Thus, the sales data in Feb will be 0.  
So we search the lines including "Accumulated" and double the data in cols "Feb".


In [15]:
#filter the rows containing "Accumulated"
accumulated_data = merged_data[merged_data['Indicators'].str.contains("Accumulated")]
accumulated_data
merged_data = merged_data[~merged_data['Indicators'].isin(accumulated_data['Indicators'])]

In [16]:
# Find all Feb cols
feb_columns = [col for col in accumulated_data.columns if 'Feb' in col]

# Double the Feb data
for col in feb_columns:
    accumulated_data[col] *= 2

print(accumulated_data[feb_columns].head())


   Feb 2023  Feb 2022  Feb 2021  Feb 2020  Feb 2019
1   77067.0   74426.0   69736.8   52129.8   66064.0
3       3.5       6.7      33.8     -20.5       8.2
5   28485.8   27229.6   24374.5   16949.8   23096.0
7       2.1       9.1      43.9     -23.4       3.5
9   66796.3   64593.1   60551.6   44881.1   56210.0


In [19]:
# Merge
merged_data = pd.concat([merged_data, accumulated_data]).reset_index(drop=True)
output_file = 'NBS_merged.xlsx'
with pd.ExcelWriter(output_file) as writer:
    merged_data.to_excel(writer, index=False, sheet_name='Merged')

print(f'Sheets have been merged and saved to {output_file}')

Sheets have been merged and saved to NBS_merged.xlsx
